<a href="https://colab.research.google.com/github/GenericP3rson/Pokemon-Notebooks/blob/main/Pok%C3%A9GSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SELECT Statement With Pokémon
Gotta Catch 'Em All!

Now that you have your data loaded, the next big step will be catching them all and extracting relevant information from your graph using queries.

This lesson will walk you through the basics of the SELECT statement with the Pokémon data.

Phrases:
- [ ] SELECT
- [ ] WHERE
- [ ] ACCUM
- [ ] POST-ACCUM
- [ ] HAVING
- [ ] ORDER BY
- [ ] LIMIT

Reference: https://docs.tigergraph.com/dev/gsql-ref/querying/select-statement

# Part I: Connect with the Graph
Here, install pyTigerGraph and connect to your graph. If you are running into problems, be sure that your solution is running!

In [ ]:
!pip install pyTigerGraph

import pyTigerGraph as tg
import json # To print data out nicely

In [ ]:
conn = tg.TigerGraphConnection(host="https://shreya.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True) # Here, change the subdomain to your subdomain!

conn.graphname = "PokemonGraph"
conn.apiToken = conn.getToken(conn.createSecret())

# Step II: Write Your First Query!
Faithful to our sport, let's catch 'em all! To start, let's create a seed. 

In [ ]:
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH PokemonGraph { 
  catchThemAll = {Pokemon.*}; # Pokemon.* means that it'll grab all the Pokemon
  PRINT catchThemAll; # print displays the results
}
''') # Here, we're interpreting a query. 
# catchThemAll is our Seed, or similar to a starting point. 
# Pokemon.* means that we're collecting all vertices of type Pokemon
# PRINT catchThemAll; will print out the results (all Pokemon)

print(json.dumps(allPokemon, indent=2))

# SELECT
Nice! Next, we're going to add a SELECT statement.

In [ ]:
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH PokemonGraph { 
  catchThemAll = {Pokemon.*};
  Types = SELECT t 
          FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t; # Selects all the Types
  PRINT Types;
}
''') # One-hop query
# catchThemAll has all Pokémon
# Types selects all nodes (SELECT t) which shares an undirected edge from any Pokémon (FROM catchThemAll:s-). 
# The undirected edge must be POKEMON_TYPE and it will return a Type vertex ((POKEMON_TYPE:e)-Type:t;)

# Let's break it up again. For Types, at the end, we define the Type is t (Type:t). 
# So, at the start of our function, we tell it to SELECT t
# Next, how will we get to Type t? Well, we'll use our values from catchThemAll, which we defined as s (catchThemAll:s), 
# then crawl across the POKEMON_TYPE edge, which we defined as e (POKEMON_TYPE:e). 
# The - means undirected. If we have a directed edge, it'd be ->

# Utlimately, this is just a fancy way to print out all the Type vertices (provides every type is connected to a Pokémon)
print(json.dumps(allPokemon, indent=2))


[
  {
    "Types": [
      {
        "v_id": "bug",
        "v_type": "Type",
        "attributes": {
          "type_id": "bug"
        }
      },
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass"
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type_id": "fire"
        }
      },
      {
        "v_id": "flying",
        "v_type": "Type",
        "attributes": {
          "type_id": "flying"
        }
      },
      {
        "v_id": "water",
        "v_type": "Type",
        "attributes": {
          "type_id": "water"
        }
      },
      {
        "v_id": "poison",
        "v_type": "Type",
        "attributes": {
          "type_id": "poison"
        }
      }
    ]
  }
]


# WHERE

Now, let's make it a bit more specific. Let's filter all the types to just be the types of one Pokémon. We'll filter using the WHERE clause.

In [ ]:
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH PokemonGraph { 
  catchThemAll = {Pokemon.*};
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Selects all the types
          WHERE s.weight > 100; # Filters to just be the ones connected the pikachu
  PRINT Types;
}
''') # One-hop query
# catchThemAll has all Pokémon
# Types selects all nodes (SELECT t) which shares an undirected edge from any Pokémon (FROM catchThemAll:s-). 
# The undirected edge must be POKEMON_TYPE and it will return a Type vertex ((POKEMON_TYPE:e)-Type:t;)
# But we're filtering it: only the Pokemon (s) who have an attribute "name" of pikachu will be returned.

# Utlimately, this is prints out all the Type vertices connected to pikachu
print(json.dumps(allPokemon, indent=2))


[
  {
    "Types": [
      {
        "v_id": "flying",
        "v_type": "Type",
        "attributes": {
          "type_id": "flying"
        }
      },
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass"
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type_id": "fire"
        }
      },
      {
        "v_id": "water",
        "v_type": "Type",
        "attributes": {
          "type_id": "water"
        }
      },
      {
        "v_id": "poison",
        "v_type": "Type",
        "attributes": {
          "type_id": "poison"
        }
      }
    ]
  }
]


Instead of directly entering the Pokémon, let's next instead pass it as a parameter.

In [ ]:
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY (STRING name) FOR GRAPH PokemonGraph { 
  catchThemAll = {Pokemon.*};
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Select all connected types
          WHERE s.name == name; # Where the name of the pokemon is the inputted name
  PRINT Types;
}
''', {"name": "bulbasaur"}) # One-hop query
# At the top, we input a STRING called name
# catchThemAll has all Pokémon
# Types selects all nodes (SELECT t) which shares an undirected edge from any Pokémon (FROM catchThemAll:s-). 
# But we're filtering it: only the Pokemon (s) who have an attribute "name" of the input will be returned.

# Utlimately, this is prints out all the Type vertices connected to the inputted Pokémon
print(json.dumps(allPokemon, indent=2))


[
  {
    "Types": [
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass"
        }
      },
      {
        "v_id": "poison",
        "v_type": "Type",
        "attributes": {
          "type_id": "poison"
        }
      }
    ]
  }
]


# ACCUM 

Awesome! Next, let's find the average weight of the Pokémon by type. We'll use an Accumulator for this.

There are a bunch of Accumulators you can use! For ours, we'll use an AvgAccum (average), but here are possible options:
1. **SumAccum\<int>** stores the running total of all numbers passed.
2. **MaxAccum\<int>** stores the maximum number passed.
3. **MinAccum\<int>** stores the minimum number passed.
4. **AvgAccum** stores the running average of all numbers passed.
5. **SetAccum\<int>** requires that all elements are unique.
6. **ListAccum\<int>** has a list of the elements in the order it was passed in.
7. **MapAccum<int, SumAccum\<int>>** use key-value pairs to store data like a map.
8. **HeapAccum\<Tuple>** stores the data in tuples in ascending or descending order. One can also set a capacity limit; when the limit is reached, it will either remove the highest or lowest value.

In [ ]:
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY ( ) FOR GRAPH PokemonGraph { 
  AvgAccum @weights; 
  catchThemAll = {Pokemon.*}; # All Pokemon vertices
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Grab all the Type vertices
          ACCUM t.@weights += s.weight; # Find the Average Weight for each type (this will be printed as an attribute of type)
  PRINT Types;
}
''', {}) # One-hop query
# catchThemAll has all Pokémon
# Types selects all nodes (SELECT t) which shares an undirected edge from any Pokémon (FROM catchThemAll:s-). 
# Next, it attaches the weights accumulator to the type (with the t.@weights). It will find the average weight for all the Pokémon connected to each Type.
# When it prints Type, notice that @weights is an attribute!
# Also, for the prefix, @ means local and @@ means global.

# Utlimately, this is just a fancy way to print out all the Type vertices (provides every type is connected to a Pokémon)
print(json.dumps(allPokemon, indent=2))


[
  {
    "Types": [
      {
        "v_id": "flying",
        "v_type": "Type",
        "attributes": {
          "type_id": "flying",
          "@weights": 414.33333
        }
      },
      {
        "v_id": "water",
        "v_type": "Type",
        "attributes": {
          "type_id": "water",
          "@weights": 390
        }
      },
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass",
          "@weights": 399.66667
        }
      },
      {
        "v_id": "poison",
        "v_type": "Type",
        "attributes": {
          "type_id": "poison",
          "@weights": 271
        }
      },
      {
        "v_id": "bug",
        "v_type": "Type",
        "attributes": {
          "type_id": "bug",
          "@weights": 145.83333
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type_id": "fire",
          "@weights": 393.33333
        }
      },
      {
 

In [ ]:
# CHALLENGE: Try to modify the query but find the Pokémon who are tallest
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH PokemonGraph { 
  AvgAccum @heights; 
  catchThemAll = {Pokemon.*}; # All Pokemon vertices
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Grab all the Type vertices
          ACCUM t.@heights += s.height; # Find the Average Weight for each type (this will be printed as an attribute of type)
  PRINT Types;
}
''', {}) 

print(json.dumps(allPokemon, indent=2))

[
  {
    "Types": [
      {
        "v_id": "flying",
        "v_type": "Type",
        "attributes": {
          "type_id": "flying",
          "@heights": 11.4
        }
      },
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass",
          "@heights": 12.33333
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type_id": "fire",
          "@heights": 11.33333
        }
      },
      {
        "v_id": "normal",
        "v_type": "Type",
        "attributes": {
          "type_id": "normal",
          "@heights": 9.66667
        }
      },
      {
        "v_id": "poison",
        "v_type": "Type",
        "attributes": {
          "type_id": "poison",
          "@heights": 9.33333
        }
      },
      {
        "v_id": "bug",
        "v_type": "Type",
        "attributes": {
          "type_id": "bug",
          "@heights": 6.66667
        }
      },
      {


# POST-ACCUM
The POST-ACCUM is nearly the same thing as the ACCUM clause, but it always follows the ACCUM clause and allows you to use variables from the ACCUM clause


In [ ]:
# Let's try to find the biggest average weight.  
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY (STRING name) FOR GRAPH PokemonGraph { 
  AvgAccum @weights; 
  MaxAccum<double> @@biggestAvgWeight; # MaxAccum
  catchThemAll = {Pokemon.*}; # All Pokemon vertices
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Grab all the Type vertices
          ACCUM t.@weights += s.weight # Find the Average Weight for each type (this will be printed as an attribute of type)
          POST-ACCUM @@biggestAvgWeight += t.@weights; # Finds the biggest average weight
  PRINT Types;
  PRINT @@biggestAvgWeight;
}
''', {"name": "pikachu"}) 

print(json.dumps(allPokemon, indent=2))

[
  {
    "Types": [
      {
        "v_id": "grass",
        "v_type": "Type",
        "attributes": {
          "type_id": "grass",
          "@weights": 399.66667
        }
      },
      {
        "v_id": "water",
        "v_type": "Type",
        "attributes": {
          "type_id": "water",
          "@weights": 390
        }
      },
      {
        "v_id": "bug",
        "v_type": "Type",
        "attributes": {
          "type_id": "bug",
          "@weights": 145.83333
        }
      },
      {
        "v_id": "flying",
        "v_type": "Type",
        "attributes": {
          "type_id": "flying",
          "@weights": 387.6
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type_id": "fire",
          "@weights": 393.33333
        }
      },
      {
        "v_id": "normal",
        "v_type": "Type",
        "attributes": {
          "type_id": "normal",
          "@weights": 186.6
        }
      },
      {
   

# HAVING

HAVING is similar to the WHERE clause. The main difference is that it is applied after the ACCUM and POST-ACCUM clauses. This means that you can use the accumulator's variables in this clause.


In [ ]:
# Here, we'll find all the Pokémon Types with an average weight over 300.
allPokemon = conn.runInterpretedQuery('''
INTERPRET QUERY (STRING name) FOR GRAPH PokemonGraph { 
  AvgAccum @weights; 
  catchThemAll = {Pokemon.*}; # All Pokemon vertices
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Grab all the Type vertices
          ACCUM t.@weights += s.weight # Find the Average Weight for each type (this will be printed as an attribute of type)
          HAVING t.@weights >= 300; # Further filters the types based on their weights; it must be greater than or equal to 300.
  PRINT Types;
}
''', {"name": "pikachu"}) # One-hop query
# catchThemAll has all Pokémon
# Types selects all nodes (SELECT t) which shares an undirected edge from any Pokémon (FROM catchThemAll:s-). 
# Next, it attaches the weights accumulator to the type (with the t.@weights). It will find the average weight for all the Pokémon connected to each Type.
# When it prints Type, notice that @weights is an attribute!
# Also, for the prefix, @ means local and @@ means global.

# Utlimately, this is just a fancy way to print out all the Type vertices (provides every type is connected to a Pokémon)
print(json.dumps(allPokemon, indent=2))


# ORDER BY

Next, what if we don't want to order the list we returned? We can do that using ORDER BY. 

This, however, is a special query because we can't interpret it: we need to install it into our graph. 

In [ ]:
# Let's order the Pokémon Types from highest to lowest weight

allPokemon = conn.gsql('''
DROP QUERY orderTypeByPokemonCount; # Removes the query if it already exists
CREATE QUERY orderTypeByPokemonCount() FOR GRAPH PokemonGraph { 
  AvgAccum @weights; # Our AvgAccum
  catchThemAll = {Pokemon.*}; # All the Pokémon
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Type vertices connect to Pokemone with POKEMON_TYPE
          ACCUM t.@weights += s.weight # Add to the AvgAccum
          ORDER BY t.@weights DESC; # Order them descending
  PRINT Types;
}
INSTALL QUERY orderTypeByPokemonCount;
''') 
allPokemon = conn.runInstalledQuery("orderTypeByPokemonCount") # Runs the query

print(json.dumps(allPokemon, indent=2))


[
  {
    "Types": [
      {
        "v_id": "rock",
        "v_type": "Type",
        "attributes": {
          "type": "rock",
          "@weights": 1587.5
        }
      },
      {
        "v_id": "ice",
        "v_type": "Type",
        "attributes": {
          "type": "ice",
          "@weights": 1262.5
        }
      },
      {
        "v_id": "ground",
        "v_type": "Type",
        "attributes": {
          "type": "ground",
          "@weights": 832.6
        }
      },
      {
        "v_id": "fighting",
        "v_type": "Type",
        "attributes": {
          "type": "fighting",
          "@weights": 556.66667
        }
      },
      {
        "v_id": "water",
        "v_type": "Type",
        "attributes": {
          "type": "water",
          "@weights": 515.33333
        }
      },
      {
        "v_id": "fire",
        "v_type": "Type",
        "attributes": {
          "type": "fire",
          "@weights": 496.44444
        }
      },
      {
        "v_id":

# LIMIT
LIMIT simply restrains the number of resulting values.

In [ ]:
# Let's grab the top five most heavy Pokémon

allPokemon = conn.gsql('''
DROP QUERY orderTypeByPokemonCount; # Removes the query if it already exists
CREATE QUERY orderTypeByPokemonCount() FOR GRAPH PokemonGraph { 
  AvgAccum @weights; # Our AvgAccum
  catchThemAll = {Pokemon.*}; # All the Pokémon
  Types = SELECT t FROM catchThemAll:s- (POKEMON_TYPE:e)-Type:t # Type vertices connect to Pokemone with POKEMON_TYPE
          ACCUM t.@weights += s.weight # Add to the AvgAccum
          ORDER BY t.@weights DESC # Order them descending
          LIMIT 5; # Limits the results to just the top five
  PRINT Types;
}
INSTALL QUERY orderTypeByPokemonCount;
''') 
allPokemon = conn.runInstalledQuery("orderTypeByPokemonCount") # Runs the query

print(json.dumps(allPokemon, indent=2))


# That's It!

Great job! You've now learned the basics of GSQL! Now, go ahead and write your own queries. It's your turn to catch 'em all!